In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

import deeplabcut


Autosaving every 180 seconds


In [45]:
# READ X,Y and Likelihoods from a presaved CSV file
   
def load_csv(fname):
    with open(fname, newline='') as csvfile:
        data = list(csv.reader(csvfile))

    labels = data[1]

    # load values
    data_array = np.array(data[3:])

    # 
    #labels = ['fnose','f_leye','f_reye','f_lear','f_rear','f_',
    #         'male_nose','male_left_ear','male_right_ear','male_base_of_tail',
    #          'pup_shaved_nose','pup_shaved_left_ear','pup_shaved_right_ear','pup_shaved_base_of_tail',
    #          'pup_noshave_nose','pup_noshave_left_ear','pup_noshave_right_ear','pup_noshave_base_of_tail'             
    #         ]
    
    labels = labels[1:]
    
    traces = []
    traces_nan = []
    for idx in range(1,len(labels)-1,3):
        temp = data_array[1:,idx:idx+3]
        idx1 = np.where(temp=='')[0]
        temp[idx1]=0
        temp = temp.astype(np.float)# np.array(temp)

        # replace low likelihoods with median
        likelihoods = temp[:,2]
        idx2 = np.where(likelihoods<0.8)[0]
        temp[idx2,0]=np.median(temp[:,0])
        temp[idx2,1]=np.median(temp[:,1])
        traces.append(temp.copy())
        
        temp[idx2,0]=np.nan
        temp[idx2,1]=np.nan
        traces_nan.append(temp.copy())

    return traces, labels, traces_nan

def load_csv_data(data_all):
    traces = np.array(data_all[0])
    #print (traces[0].shape)
    tracesx = []
    tracesy = []
    likelihoods = []

    for k in range(0,len(traces),1):
        tracesx.append(traces[k][:,0])
        tracesy.append(traces[k][:,1])
        likelihoods.append(traces[k][:,2])

    likelihoods = np.array(likelihoods)
    tracesx = np.array(tracesx)
    tracesy = np.array(tracesy)
    print ("tracex: ", tracesx.shape)
    print ("tracey: ", tracesy.shape)
    print ("likelihood: ", likelihoods.shape)
        
    return tracesx, tracesy, likelihoods

# MAKE VIDOES + LABELS 

# Robust outlier detection function to crop some of the DLC errors;
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

In [46]:
# load a csv file; 

# OLD CSV DATA
if False:
    csv_fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx.csv'
    data_all = load_csv(csv_fname)
    tracesx, tracesy, likelihoods = load_csv_data(data_all)
# NEW INFERENCE FIXED DATA
else:
    # alterantive data:
    csv_fname= '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_inference_fixed.npz'
    traces = np.load(csv_fname)
    tracesx = traces['tracesx']
    tracesy = traces['tracesy']
    likelihoods = traces['probs']
    print ("tracesx: ", tracesx.shape)

    
#traces_all = np.stack((np.stack((tracesx,tracesy)),likelihoods))
print ("DONE Loading traces and likelihoods")

tracesx:  (56, 89989)
DONE Loading traces and likelihoods


In [33]:
# Save median/single feature trackelts and likelihoods
# initialize start and ends of video to be analyzed
start = 0
end = tracesx.shape[1]
#end = 1000

tracesx_mean =[]
tracesy_mean =[]
likelihoods_mean =[]
threshold = 0.01

total_nans = [0]*4

for k in range(start,end,1):    
    if k%5000==0:
        print (k, "/", end)
        
    tracesx_mean.append([])
    tracesy_mean.append([])
    # PLOT MEAN FEATURE
    for p in range(0, tracesx.shape[0], 14):
        l = likelihoods[p:p+14,k]
        idx = np.where(l<threshold)[0]
        #print(idx.shape)

        traces_local = tracesx[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        
        # remove nans from data
        idx_nan = np.where(np.isnan(traces_local)==True)[0]
        traces_local = np.delete(traces_local, idx_nan)

        # reject outliers
        traces_local = reject_outliers(traces_local, m = 4)

        x = np.nanmedian(traces_local)
        
        
        traces_local = tracesy[p:p+14,k]
        traces_local = np.delete(traces_local, idx)

        # remove nans from data
        idx_nan = np.where(np.isnan(traces_local)==True)[0]
        traces_local = np.delete(traces_local, idx_nan)

        # reject outliars
        traces_local = reject_outliers(traces_local, m = 4)
        y = np.nanmedian(traces_local)
        #print ("XY: ", x,y)
        tracesx_mean[k].append(x)
        tracesy_mean[k].append(y)
        
        if np.isnan(x) or np.isnan(y):
            total_nans[p//14]+=1

tracesx_mean = np.array(tracesx_mean)
tracesy_mean = np.array(tracesy_mean)

print ("Total nans (missing frames) for each animal: ", total_nans)

print ("DONE Loading mean traces")
np.savez(csv_fname[:-4]+'_single_feature_traces_WITH_INFERENCES.npz', 
         tracesx=tracesx_mean,
         tracesy=tracesy_mean)

[0, 0, 0, 0]
0 / 89989
5000 / 89989
10000 / 89989
15000 / 89989
20000 / 89989
25000 / 89989
30000 / 89989
35000 / 89989
40000 / 89989
45000 / 89989
50000 / 89989
55000 / 89989
60000 / 89989
65000 / 89989
70000 / 89989
75000 / 89989
80000 / 89989
85000 / 89989
total nans: [31993, 44883, 64551, 77055]
DONE Loading mean traces


Total nans (missing frames) for each animal:  [31993, 44883, 64551, 77055]
DONE Loading mean traces


In [27]:
print ("DONE Loading mean traces")
np.savez('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_single_feature_traces_WITH_INFERENCE.npz', 
        tracesx=tracesx_mean,
        tracesy=tracesy_mean)

DONE Loading mean traces


In [39]:
print (tracesx_mean)

[[284.815     nan     nan     nan]
 [285.203     nan     nan     nan]
 [285.049     nan     nan     nan]
 ...
 [    nan     nan     nan     nan]
 [    nan     nan     nan     nan]
 [    nan     nan     nan     nan]]


In [1]:
# filter data
total=[1964, 4534, 5890, 9721]

In [4]:
import numpy as np
print (np.array(total)/16572)

[0.11851315 0.27359401 0.35541878 0.58659184]
